# Re-implementation of AudioLM

Lorem ipsum dolor sit amet, consectetur adipiscing elit. Morbi quis eros fringilla, tristique sem non, dapibus urna. Integer scelerisque porta nulla eget lobortis. Donec quis malesuada est. Phasellus porta risus non nulla egestas dapibus. Cras vehicula in sapien in tempus. Maecenas dui odio, congue eget aliquam eget, congue ut eros. Donec elementum pretium iaculis. Ut eu purus et tortor faucibus pulvinar. Sed feugiat bibendum metus a rhoncus.

Pellentesque nunc dolor, tincidunt ut leo condimentum, fermentum dictum nisl. Pellentesque viverra diam sit amet augue gravida tristique. Integer varius finibus tortor in cursus. Donec faucibus tristique risus. Pellentesque mattis enim et odio pulvinar euismod. Sed dignissim dolor sit amet risus mattis, eget iaculis odio tincidunt. Morbi id commodo sem, quis euismod purus. Nunc volutpat vitae erat id dictum. Pellentesque ac tincidunt est. Sed in enim vitae felis tempor laoreet. Donec sit amet facilisis quam. Etiam vitae facilisis odio, nec malesuada massa. Praesent dignissim, leo cursus lacinia vehicula, diam ipsum iaculis velit, sed volutpat tellus risus non elit. Nunc non finibus nibh. 

## Import and settings

In [ ]:
# missing imports

In [ ]:
seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)  # If using CUDA
np.random.seed(seed)
random.seed(seed)

## Semantic tokens with w2v-BERT-like model

In [ ]:
from hubertKM import SemanticTokenizer, visualizeEmbeddings

In [ ]:
w2vBERT = SemanticTokenizer("facebook/hubert-base-ls960","./hubertKM/hubert_base_ls960_L9_km500.bin") 

In [ ]:
## missing model Test

In [ ]:
## missing visualization

## Acoustic tokens with SoundStream codec model

In [ ]:
from SoundStream import soundstream_16khz

In [ ]:
soundStream = soundstream_16khz()

In [ ]:
## missing model Test

## Token Dataset creation

In [ ]:
from data import storeTokens, TokensDataset, store_from_librilight

In [ ]:
tokenPath = "out" ## output file directory
tokenFile = "tokens.csv" ## output file name
audioPath = "data" ## data_location

In [ ]:
#fileCount = storeTokens(audioPath, tokenPath, tokenFile, w2vBERT, soundStream, fileCountCheckpoint = 10)
fileCount = store_from_librilight(tokenPath, tokenFile, w2vBERT, soundStream, fileCountCheckpoint = 10, subset = "10h")

In [ ]:
AUDIO_LENGTH = 30
CROP_LENGTH = [15,5,1]

semanticDataset = TokensDataset(tokenPath, tokenFile, mode = "semantic", expected_audio_length = AUDIO_LENGTH, crop_length = CROP_LENGTH)
coarseDataset = TokensDataset(tokenPath, tokenFile, mode = "coarse", expected_audio_length = AUDIO_LENGTH, crop_length = CROP_LENGTH)
fineDataset = TokensDataset(tokenPath, tokenFile, mode = "fine", expected_audio_length = AUDIO_LENGTH, crop_length = CROP_LENGTH)

## AudioLM: a transformer-based audio model

### Semantic Transformer: expanding a sentence snippet

In [ ]:
TRAINING_PERCENTAGE = 0.8
BATCH_SIZE = 16

train_dataset, valid_dataset = random_split(coarseDataset, [TRAINING_PERCENTAGE, 1 - TRAINING_PERCENTAGE])
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False)

trainer = pl.Trainer(
    max_epochs=30,
    accelerator='gpu' if torch.cuda.is_available() else 'cpu',
    log_every_n_steps=1,
    #devices=1 if torch.cuda.is_available() else None
)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

checkpoint_path = "checkpoints/semantic-checkpoint-10h-colab.ckpt"
d_model=256 #1024
num_layers=3 #12
num_heads=4 #16
dim_feedforward=1024 #4096
audioDuration=15 #30
vocab_size=500


if os.path.exists(checkpoint_path):
    print(f"Checkpoint found at {checkpoint_path}. Resuming old model...")
    model = SemanticTransformer.load_from_checkpoint(checkpoint_path, d_model=d_model, num_layers = num_layers, num_heads=num_heads, k = int(d_model/num_heads), dim_feedforward=dim_feedforward, audioDuration = audioDuration, vocab_size = vocab_size, myDevice = device)
else:
    print("No checkpoint found. Starting from scratch...")
    model = SemanticTransformer(d_model=d_model, num_layers = num_layers, num_heads=num_heads, k = int(d_model/num_heads), dim_feedforward=dim_feedforward, audioDuration = audioDuration, vocab_size = vocab_size, myDevice = device)

### Coarse Transformer: generating new audio

In [ ]:
TRAINING_PERCENTAGE = 0.8
BATCH_SIZE = 16

train_dataset, valid_dataset = random_split(coarseDataset, [TRAINING_PERCENTAGE, 1 - TRAINING_PERCENTAGE])
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False)

trainer = pl.Trainer(
    max_epochs=30,
    accelerator='gpu' if torch.cuda.is_available() else 'cpu',
    log_every_n_steps=1,
    #devices=1 if torch.cuda.is_available() else None
)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

checkpoint_path = "checkpoints/coarse-checkpoint-10h-colab.ckpt"
d_model=256 #1024
num_layers=3 #12
num_heads=4 #16
dim_feedforward=1024 #4096
audioDuration=5 #10
vocab_size=1024


if os.path.exists(checkpoint_path):
    print(f"Checkpoint found at {checkpoint_path}. Resuming old model...")
    model = CoarseTransformer.load_from_checkpoint(checkpoint_path, d_model=d_model, num_layers = num_layers, num_heads=num_heads, k = int(d_model/num_heads), dim_feedforward=dim_feedforward, audioDuration = audioDuration, vocab_size = vocab_size, myDevice = device)
else:
    print("No checkpoint found. Starting from scratch...")
    model = CoarseTransformer(d_model=d_model, num_layers = num_layers, num_heads=num_heads, k = int(d_model/num_heads), dim_feedforward=dim_feedforward, audioDuration = audioDuration, vocab_size = vocab_size, myDevice = device)

### Fine Transformer: improving audio quality

In [ ]:
TRAINING_PERCENTAGE = 0.8
BATCH_SIZE = 16

train_dataset, valid_dataset = random_split(coarseDataset, [TRAINING_PERCENTAGE, 1 - TRAINING_PERCENTAGE])
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False)

trainer = pl.Trainer(
    max_epochs=30,
    accelerator='gpu' if torch.cuda.is_available() else 'cpu',
    log_every_n_steps=1,
    #devices=1 if torch.cuda.is_available() else None
)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

checkpoint_path = "checkpoints/fine-checkpoint-10h-colab.ckpt"
d_model=256 #1024
num_layers=3 #12
num_heads=4 #16
dim_feedforward=1024 #4096
audioDuration=1.5 #3
vocab_size=1024


if os.path.exists(checkpoint_path):
    print(f"Checkpoint found at {checkpoint_path}. Resuming old model...")
    model = FineTransformer.load_from_checkpoint(checkpoint_path, d_model=d_model, num_layers = num_layers, num_heads=num_heads, k = int(d_model/num_heads), dim_feedforward=dim_feedforward, audioDuration = audioDuration, vocab_size = vocab_size, myDevice = device)
else:
    print("No checkpoint found. Starting from scratch...")
    model = FineTransformer(d_model=d_model, num_layers = num_layers, num_heads=num_heads, k = int(d_model/num_heads), dim_feedforward=dim_feedforward, audioDuration = audioDuration, vocab_size = vocab_size, myDevice = device)

## Inference and results